<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Javier Santidrián Salas
- Nombre de alumno 2: Patricio Yáñez Alarcón


### **Link de repositorio de GitHub:** https://github.com/jsantidrian/labs-Santidrian-Yanez

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# Clasificadores/Regresores
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import r2_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 78.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df_players = pd.read_csv('/content/drive/My Drive/U/11 Semestre/Lab Progra/Labs/Lab 8/stats_players.csv')
df_salarios = pd.read_csv('/content/drive/My Drive/U/11 Semestre/Lab Progra/Labs/Lab 8/salarios.csv')

Visualizamos:

In [ ]:
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


In [ ]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17588 entries, 0 to 17587
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               17588 non-null  object
 1   Nationality        17588 non-null  object
 2   National_Position  1075 non-null   object
 3   Club_Position      17587 non-null  object
 4   Height             17588 non-null  int64 
 5   Weight             17588 non-null  int64 
 6   Preffered_Foot     17588 non-null  object
 7   Age                17588 non-null  int64 
 8   Work_Rate          17588 non-null  object
 9   Weak_foot          17588 non-null  int64 
 10  Skill_Moves        17588 non-null  int64 
 11  Ball_Control       17588 non-null  int64 
 12  Dribbling          17588 non-null  int64 
 13  Marking            17588 non-null  int64 
 14  Sliding_Tackle     17588 non-null  int64 
 15  Standing_Tackle    17588 non-null  int64 
 16  Aggression         17588 non-null  int64

In [ ]:
df_salarios

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


In [ ]:
df_salarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2534 non-null   int64  
 1   Player         2534 non-null   object 
 2   Weekly Salary  2534 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.5+ KB


## 1.1 Predicción de Seleccionados Nacionales

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país.

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

Antes de separar en características X y variable target y, dropearemos el único dato que tiene un valor nulo en la variable Club_Position para evitar problemas más adelante en el resto del laboratorio (al separar en train-test):

In [ ]:
df_players.dropna(subset=['Club_Position'],inplace=True)

Obtenemos los valores de National_Position (labels de variable objetivo) y separamos el resto de características (dropearemos el nombre del jugador también, puesto que no aporta para la predicción posterior):

In [ ]:
X = df_players.drop(columns=['National_Position', 'Name'])
y = df_players['National_Position']
y

0         LS
1         RW
2         LW
3         LS
4         GK
        ... 
17583    NaN
17584    NaN
17585    NaN
17586    NaN
17587    NaN
Name: National_Position, Length: 17587, dtype: object

Valores únicos para las etiquetas:

In [ ]:
y.unique()

array(['LS', 'RW', 'LW', 'GK', 'RS', nan, 'RCB', 'LF', 'CAM', 'Sub',
       'LCB', 'RCM', 'LDM', 'LCM', 'LAM', 'CB', 'CDM', 'LM', 'RM', 'LB',
       'ST', 'CM', 'RF', 'RB', 'RDM', 'RAM', 'LWB', 'RWB'], dtype=object)

Encodearemos como clasificación binaria, asignando un 0 a los valores nan (clase negativa, los que no son seleccionados nacionales) y un 1 al resto (clase positiva, los que sí son seleccionados nacionales)

In [ ]:
# Función que asigna con un 0 a los valores nan y con un 1 al resto
def f(x):
  if pd.isnull(x):
    return 0
  else:
    return 1

# Aplicamos función a la serie de labels y
y = y.apply(f)

In [ ]:
# Visualizamos
y

0        1
1        1
2        1
3        1
4        1
        ..
17583    0
17584    0
17585    0
17586    0
17587    0
Name: National_Position, Length: 17587, dtype: int64

In [ ]:
y.value_counts()

0    16512
1     1075
Name: National_Position, dtype: int64

Hay 16.512 jugadores no seleccionados nacionales y 1075 que sí lo son

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores.

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

Primero obtengamos las variables numéricas:

In [ ]:
numerical = X.select_dtypes(include=['int', 'float']).columns
numerical

Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')

Sobre estas variables aplicaremos MinMaxScaler (para que queden normalizadas).

Ahora obtengamos las variables categóricas:

In [ ]:
categorical = X.select_dtypes(include=['object']).columns
categorical

Index(['Nationality', 'Club_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object')

Observemos los valores:

In [ ]:
X[categorical]

,Nationality,Club_Position,Preffered_Foot,Work_Rate
0,Portugal,LW,Right,High / Low
1,Argentina,RW,Left,Medium / Medium
2,Brazil,LW,Right,High / Medium
3,Uruguay,ST,Right,High / Medium
4,Germany,GK,Right,Medium / Medium
...,...,...,...,...
17583,Republic of Ireland,Sub,Right,Medium / Medium
17584,Republic of Ireland,Sub,Right,Medium / Medium
17585,England,Res,Right,High / Medium
17586,Scotland,Sub,Right,Medium / Medium


Del reporte en HTML se aprecia que:

- Hay 160 categorías nominales distintas en la variable Nationality
- Hay 29 categorías nominales distintas en la variable Club_Position
- Hay 2 categorías nominales distintas en la variable Preffered_Foot
- Hay 9 categorías nominales distintas en la variable Work_Rate

Sobre estas variables aplicaremos OneHotEncoding (dada su naturaleza nominal).

Preprocesamiento:

In [ ]:
# Definimos las transformaciones para cada columna:
# MinMaxScaler para las numéricas, OneHotEncoder para las categóricas
preprocesamiento = ColumnTransformer(
    transformers=[
        ('numerical', MinMaxScaler(), numerical),
        ('categorical', OneHotEncoder(sparse=False,handle_unknown='ignore'), categorical)
    ]
)

Crearemos 3 pipelines con 3 clasificadores de acuerdo a la guía de sklearn: Linear SVC (debido a que tenemos menos de 100 mil datos), KNeighbors Classifier (debido a que no trabajamos con NLP) y Random Forest (Ensemble Classifier, se elige también pues no trabajamos con NLP):

In [ ]:
# Pipeline de Linear SVC
pipeline_lsvc = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('classifier', LinearSVC(random_state=42))
])

# Pipeline de KNeighbors Classifier
pipeline_knn = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('classifier', KNeighborsClassifier())
])

# Pipeline de Random Forest
pipeline_rf = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('classifier', RandomForestClassifier(random_state=42))
])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

Dividimos en train-test en proporción 80-20 de manera estratificada (es decir manteniendo la distribución original de labels de cada clase en ambos conjuntos):

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Entrenemos los pipelines, es decir, preprocesaremos el conjunto de train (primer paso del pipeline) y luego ajustaremos el clasificador respectivo a aquellos datos (segundo paso del pipeline) para calcular predicciones sobre el conjunto de test:

In [ ]:
# Linear SVC
pipeline_lsvc.fit(X_train, y_train)
y_pred_lsvc = pipeline_lsvc.predict(X_test)

# KNN
pipeline_knn.fit(X_train,y_train)
y_pred_knn = pipeline_knn.predict(X_test)

# Random Forest
pipeline_rf.fit(X_train,y_train)
y_pred_rf = pipeline_rf.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Imprimimos resultados:

In [ ]:
print('Linear SVC:\n')
print(classification_report(y_test, y_pred_lsvc))
print('\nKNN:\n')
print(classification_report(y_test, y_pred_knn))
print('\nRandom Forest:\n')
print(classification_report(y_test, y_pred_rf))

Linear SVC:

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3303
           1       0.74      0.39      0.51       215

    accuracy                           0.95      3518
   macro avg       0.85      0.69      0.74      3518
weighted avg       0.95      0.95      0.95      3518


KNN:

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.56      0.07      0.12       215

    accuracy                           0.94      3518
   macro avg       0.75      0.53      0.54      3518
weighted avg       0.92      0.94      0.92      3518


Random Forest:

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      3303
           1       0.69      0.12      0.20       215

    accuracy                           0.94      3518
   macro avg       0.82      0.56      0.58      3518
weighted avg       0.93      0.94    

**Respuesta**:

El rendimiento de cada modelo es analizado mediante classification_report, una función de sklearn que proporciona un informe completo de las métricas de evaluación, donde se encuentran métricas tales como:

- Accuracy: Representa la proporción de muestras clasificadas correctamente sobre el total de muestras evaluadas.
- Precision: Representa la proporción de verdaderos positivos (muestras correctamente clasificadas como positivas) sobre el total de muestras clasificadas como positivas por el modelo. Un valor cercano a 1 minimiza los falsos positivos.
- Recall: Representa la proporción de verdaderos positivos (muestras correctamente clasificadas como positivas) sobre el total de muestras positivas en el conjunto de datos. Un valor cercano a 1 minimiza los falsos negativos.
- F1-score: Representa un equilibrio entre precision y recall (es la media armónica entre los dos valores). Es útil cuando hay un desequilibrio de clases.

Los valores de estas 3 últimas métricas son entregados en 3 formatos:

- Por clase (es decir, considerando a 0 como la clase positiva y luego considerando a 1 como la clase positiva).
- Como macro average (media no ponderada, toma el promedio de las precisiones de ambas clases, independientemente de su tamaño o distribución en los datos, donde las dos clases tienen el mismo peso en el cálculo de la media).
- Como weighted average (media ponderada, toma el promedio ponderado de las precisiones de ambas clases, teniendo en cuenta el número de muestras en cada clase, donde la clase con más muestras tiene un mayor impacto en el cálculo de la media ponderada).

Además entrega una medida del support (es decir, la cantidad de datos por cada clase y la cantidad total de datos, todo en el conjunto de test).

Ahora, respecto a los resultados obtenidos para los 3 clasificadores tenemos que:

- Son muy buenos los valores de accuracy (0.95 en Linear SVC, 0.94 en KNN y 0.94 en Random Forest). Esto implica que los modelos tienen un buen rendimiento en la clasificación de las muestras, acertando en la mayoría de las predicciones.

- Los valores de precision y recall para la clase positiva 1 (la que indica que un jugador es seleccionado nacional) no son muy buenos en general (para precision: 0.74 en Linear SVC, 0.56 en KNN y 0.69 en Random Forest, mientras que para recall: 0.39 en Linear SVC, 0.07 en KNN y 0.12 en Random Forest). Estos bajos valores de precision y recall para la clase positiva 1 indica que el modelo tiene dificultades para identificar y clasificar correctamente los ejemplos relevantes de aquella clase (es decir predecir si un jugador será seleccionado nacional), cometiendo errores tanto de falsos positivos (debido a precision, pero no tantos pues no es tan cercano a 0) como de falsos negativos (debido al bajo recall, donde aquí hay una mayor cantidad de errores pues los valores son más cercanos a 0).

- Debido a la cantidad desbalanceada de ejemplos por clase, los resultados anteriores podrían mejorarse haciendo oversampling de la clase minoritaria (es decir la clase positiva 1 que indica que un jugador es seleccionado nacional) en el conjunto de train, por ejemplo mediante SMOTE, y entrenar con esos nuevos datos los modelos. Luego las métricas obtenidas tras predecir en el conjunto de test deberían ser distintas y más balanceadas entre clases.



## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.
- [ ] Comente los resultados obtenidos.

**Respuesta:**

Obtenemos los valores de Club_Position (labels de variable objetivo) y separamos el resto de características (dropearemos el nombre del jugador también, puesto que no aporta para la predicción posterior):

In [ ]:
X = df_players.drop(columns=['Club_Position', 'Name'])
y = df_players['Club_Position']
y

0         LW
1         RW
2         LW
3         ST
4         GK
        ... 
17583    Sub
17584    Sub
17585    Res
17586    Sub
17587    Sub
Name: Club_Position, Length: 17587, dtype: object

Valores únicos para las etiquetas:

In [ ]:
y.unique()

array(['LW', 'RW', 'ST', 'GK', 'Sub', 'RCM', 'CAM', 'LCB', 'LCM', 'RS',
       'RB', 'RCB', 'LM', 'LDM', 'RM', 'LB', 'CDM', 'RDM', 'LF', 'CB',
       'LAM', 'Res', 'CM', 'LS', 'RF', 'RWB', 'RAM', 'LWB', 'CF'],
      dtype=object)

Encodearemos las etiquetas según lo pedido por Renacín, donde primero eliminaremos todos los datos que tengan otras clases distintas a las nombradas:

In [ ]:
# Lista de clases permitidas en Club_Position
clases_permitidas = ['ST',
                     'CF',
                     'RW',
                     'CAM',
                     'LW',
                     'RM',
                     'CM',
                     'LM',
                     'RWB',
                     'CDM',
                     'LWB',
                     'RB',
                     'CB',
                     'LB',
                     'GK']

# Datos filtrados
y = y[y.isin(clases_permitidas)]
X = X.loc[y.index]

In [ ]:
# Función que hace el mapeo correspondiente de las etiquetas
def g(x):
  if x in ['ST','CF']:
    return 'ataque'
  elif x in ['RW','CAM','LW']:
    return 'central_ataque'
  elif x in ['RM', 'CM', 'LM']:
    return 'central'
  elif x in ['RWB', 'CDM', 'LWB']:
    return 'central_defensa'
  elif x in ['RB', 'CB', 'LB']:
    return 'defensa'
  elif x in ['GK']:
    return 'arquero'

# Aplicamos función a la serie de labels y
y = y.apply(g)

In [ ]:
# Visualizamos
y

0        central_ataque
1        central_ataque
2        central_ataque
3                ataque
4               arquero
              ...      
17298           defensa
17301           defensa
17310           central
17312           defensa
17442           defensa
Name: Club_Position, Length: 3939, dtype: object

Conteo de datos por cada clase:

In [ ]:
y.value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

Para hacer el preprocesamiento, primero obtengamos las variables numéricas:

In [ ]:
numerical = X.select_dtypes(include=['int', 'float']).columns
numerical

Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')

Sobre estas variables aplicaremos MinMaxScaler (para que queden normalizadas).

Ahora obtengamos las variables categóricas:

In [ ]:
categorical = X.select_dtypes(include=['object']).columns
categorical

Index(['Nationality', 'National_Position', 'Preffered_Foot', 'Work_Rate'], dtype='object')

Observemos los valores:

In [ ]:
X[categorical]

,Nationality,National_Position,Preffered_Foot,Work_Rate
0,Portugal,LS,Right,High / Low
1,Argentina,RW,Left,Medium / Medium
2,Brazil,LW,Right,High / Medium
3,Uruguay,LS,Right,High / Medium
4,Germany,GK,Right,Medium / Medium
...,...,...,...,...
17298,Norway,NaN,Right,Medium / Medium
17301,Republic of Ireland,NaN,Left,Medium / Medium
17310,England,NaN,Right,Medium / Medium
17312,England,NaN,Right,Medium / Medium


Del reporte en HTML se aprecia que:

- Hay 160 categorías nominales distintas en la variable Nationality
- Hay 27 categorías nominales distintas en la variable National_Position
- Hay 2 categorías nominales distintas en la variable Preffered_Foot
- Hay 9 categorías nominales distintas en la variable Work_Rate

Sobre estas variables aplicaremos OneHotEncoding (dada su naturaleza nominal).

Preprocesamiento:

In [ ]:
# Definimos las transformaciones para cada columna:
# MinMaxScaler para las numéricas, OneHotEncoder para las categóricas
preprocesamiento = ColumnTransformer(
    transformers=[
        ('numerical', MinMaxScaler(), numerical),
        ('categorical', OneHotEncoder(sparse=False,handle_unknown='ignore'), categorical)
    ]
)

Ahora entrenemos el nuevo pipeline y ejecutemos una nueva evaluación de este, donde el clasificador elegido es Linear SVC (pues la cantidad de datos es menor a 100 mil).

In [ ]:
# Pipeline de Linear SVC
pipeline_lsvc = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('classifier', LinearSVC(random_state=42))
])

Dividimos en train-test en proporción 80-20 de manera estratificada (es decir manteniendo la distribución original de labels de cada clase en ambos conjuntos):

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Entrenamos el pipeline, es decir, preprocesaremos el conjunto de train (primer paso del pipeline) y luego ajustaremos el clasificador a aquellos datos (segundo paso del pipeline) para calcular predicciones sobre el conjunto de test:

In [ ]:
# Linear SVC
pipeline_lsvc.fit(X_train, y_train)
y_pred_lsvc = pipeline_lsvc.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Imprimimos resultados:

In [ ]:
print('Linear SVC:\n')
print(classification_report(y_test, y_pred_lsvc))

Linear SVC:

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       0.76      0.86      0.80        86
        central       0.62      0.68      0.65       182
 central_ataque       0.52      0.31      0.39       116
central_defensa       0.41      0.17      0.24        42
        defensa       0.82      0.97      0.89       236

       accuracy                           0.75       788
      macro avg       0.69      0.66      0.66       788
   weighted avg       0.73      0.75      0.73       788



**Respuesta**:

- El valor de accuracy obtenido (0.75) no es tan bueno como antes, por lo que el modelo no predice correctamente con mucha exactitud.
- Tenemos buenas métricas de precision, recall y f1-score en general, salvo por algunas clases más desbalanceadas: la clase central_defensa y la clase central_ataque, que tienen los valores más bajos para aquellas métricas.



## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

Primero renombremos la columna "Player" de df_salarios y llamemosla "Name" (nombre del futbolista) al igual que en df_players. Además dropearemos la columna "Unnamed: 0" pues no aporta información:

In [ ]:
df_salarios.rename(columns={'Player': 'Name'}, inplace=True)
df_salarios.drop(columns=['Unnamed: 0'], inplace=True)

Ahora hagamos un inner join de la base de players y la base de sueldos (con llave primaria la columna Name) para tener las características y los sueldos para cada jugador:

In [ ]:
df = pd.merge(df_players,df_salarios,on='Name',how='inner')
df

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1248536.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1538905.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,797726.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,508923.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Phillip Menzel,Germany,NaN,Res,191,83,Right,18,Medium / Medium,3,...,40,10,25,5,10,12,8,15,5,2034.0
1862,Manuel Akanji,Switzerland,NaN,Sub,187,85,Right,21,Medium / Medium,2,...,66,51,38,25,18,26,27,40,31,54176.0
1863,Moritz Nicolas,Germany,NaN,Res,195,87,Right,19,Medium / Medium,2,...,34,16,22,9,10,17,14,17,11,2262.0
1864,Giacomo Satalino,Italy,NaN,Sub,188,70,Right,17,Medium / Medium,1,...,48,11,19,6,5,13,11,18,9,2827.0


Ahora obtenemos las etiquetas de la regresión (Weekly Salary) y separamos el resto de características (dropearemos el nombre del jugador también, puesto que no aporta para la predicción posterior):

In [ ]:
X = df.drop(columns=['Weekly Salary', 'Name'])
y = df['Weekly Salary']

Para hacer el preprocesamiento, primero obtengamos las variables numéricas:

In [ ]:
numerical = X.select_dtypes(include=['int', 'float']).columns
numerical

Index(['Height', 'Weight', 'Age', 'Weak_foot', 'Skill_Moves', 'Ball_Control',
       'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle',
       'Aggression', 'Reactions', 'Interceptions', 'Vision', 'Composure',
       'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed',
       'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy',
       'Penalties', 'Volleys'],
      dtype='object')

Sobre estas variables aplicaremos MinMaxScaler (para que queden normalizadas).

Ahora obtengamos las variables categóricas:

In [ ]:
categorical = X.select_dtypes(include=['object']).columns
categorical

Index(['Nationality', 'National_Position', 'Club_Position', 'Preffered_Foot',
       'Work_Rate'],
      dtype='object')

Del reporte en HTML se aprecia que:

- Hay 160 categorías nominales distintas en la variable Nationality
- Hay 27 categorías nominales distintas en la variable National_Position
- Hay 29 categorías nominales distintas en la variable Club_Position
- Hay 2 categorías nominales distintas en la variable Preffered_Foot
- Hay 9 categorías nominales distintas en la variable Work_Rate

Sobre estas variables aplicaremos OneHotEncoding (dada su naturaleza nominal).

Preprocesamiento:

In [ ]:
# Definimos las transformaciones para cada columna:
# MinMaxScaler para las numéricas, OneHotEncoder para las categóricas
preprocesamiento = ColumnTransformer(
    transformers=[
        ('numerical', MinMaxScaler(), numerical),
        ('categorical', OneHotEncoder(sparse=False,handle_unknown='ignore'), categorical)
    ]
)

Ahora entrenemos un pipeline con regresión lineal, donde el regresor elegido es Random Forest Regressor (pues la cantidad de datos es menor a 100 mil y no estamos seguros si pocas características serán las importantes a la hora de predecir el salario):

In [ ]:
# Pipeline de Random Forest Regressor
pipeline_rfr = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('regressor', RandomForestRegressor(random_state=42))
])

Dividimos en train-test en proporción 80-20:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20242829)

Entrenamos el pipeline, es decir, preprocesaremos el conjunto de train (primer paso del pipeline) y luego ajustaremos el regresor a aquellos datos (segundo paso del pipeline) para calcular predicciones sobre el conjunto de test:

In [ ]:
# Random Forest Regressor
pipeline_rfr.fit(X_train, y_train)
y_pred_rfr = pipeline_rfr.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Mostramos resultados de la métrica $R^2$:

In [ ]:
print('Métrica R2 para Random Forest:\n')
print(round(r2_score(y_test, y_pred_rfr),3))

Métrica R2 para Random Forest:

0.581


**Respuesta**:

- La métrica $R^2$ es una medida para evaluar la calidad de un modelo de regresión, la cual indica qué tan bien se ajustan los valores predichos por el modelo a los valores reales del conjunto de datos. Esta representa la proporción de la varianza total de la variable dependiente que es explicada por el modelo, cuyo valor fluctúa entre 0 y 1. Un valor de 0 significa que el modelo no explica nada de la variabilidad de los datos, mientras que un valor de 1 indica que el modelo explica toda la variabilidad de los datos (ajuste perfecto).

- Con Random Forest Regressor obtuvimos una métrica $R^2$ de 0.581, lo cual es justo lo solicitado por Renacín (valor superior a 0.35). Este resultado es bastante bueno.

- Para mejorar aún más la métrica $R^2$ obtenida se pueden considerar más datos (pues se captura mejor la variabilidad y los patrones subyacentes) o realizar transformaciones de las características existentes (pues pueden revelar patrones o relaciones más fuertes con la variable objetivo).

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>